# Siamese Networks for Embedding Generation

This notebook demonstrates how to build and train a Siamese Network to generate vector embeddings. Siamese Networks consist of two identical subnetworks that merge at their outputs, designed to learn embeddings so that similar items are closer in the embedding space compared to dissimilar items. This approach is particularly useful for tasks such as face recognition, signature verification, and in general, for learning a similarity metric between two inputs.

## What are Siamese Networks?

Siamese Networks are a type of neural network architecture that contain two or more identical subnetworks meant to process two separate inputs. The idea is that by comparing these inputs as they pass through the network, the model learns to distinguish between them based on their similarity. This training method allows the network to effectively generate embeddings – representations of input data in a lower-dimensional space.

## Why Generate Embeddings?

Generating embeddings can significantly enhance performance on tasks like classification, recommendation, and clustering by representing complex data (like images or text) in a form that highlights their relative similarities or differences. Embeddings make it possible to perform these tasks even with limited data, as they capture the essence of the data's features in a compressed form.

## How to Implement a Siamese Network

This notebook covers the following steps:

1. **Environment Setup**: Installation of necessary libraries (TensorFlow).
2. **Siamese Network Architecture**: Designing the architecture using TensorFlow/Keras.
3. **Data Preparation**: Preparing pairs of data for training.
4. **Model Training**: Training the Siamese Network to generate embeddings.
5. **Embedding Generation**: Using the trained network to generate vector embeddings for new data.

For more detailed information on Siamese Networks and embedding generation, the following resources might be helpful:

- Siamese Networks: [Understanding Siamese Networks](https://www.cs.cmu.edu/~rsalakhu/papers/oneshot1.pdf)
- Vector Embeddings: [Learning Embeddings](https://towardsdatascience.com/understanding-feature-vectors-extraction-cdff276b9b65)
- Embedding Generation: [Generating Embeddings with Neural Networks](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html)

By following this notebook, you'll gain a practical understanding of how to leverage Siamese Networks for embedding generation, providing a foundation for further exploration into more complex architectures and applications.


In [1]:
! pip install pandas numpy tensorflow

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend as K
import random

In [5]:
def load_pairs(dataset, num_classes):
    (train_digits, train_labels), (test_digits, test_labels) = dataset.load_data()
    # Normalize image vectors
    train_digits = train_digits.astype('float32') / 255.0
    test_digits = test_digits.astype('float32') / 255.0

    # Expand dimensions to add channel information
    train_digits = np.expand_dims(train_digits, axis=-1)
    test_digits = np.expand_dims(test_digits, axis=-1)

    # Create pairs
    digit_indices = [np.where(train_labels == i)[0] for i in range(num_classes)]
    tr_pairs, tr_y = create_pairs(train_digits, digit_indices)

    digit_indices = [np.where(test_labels == i)[0] for i in range(num_classes)]
    te_pairs, te_y = create_pairs(test_digits, digit_indices)

    return (tr_pairs, tr_y), (te_pairs, te_y)

def create_pairs(x, digit_indices):
    """Positive and negative pair creation.
    Alternates between positive and negative pairs."""
    pairs = []
    labels = []

    n = min([len(digit_indices[d]) for d in range(10)]) - 1

    for d in range(10):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, 10)
            dn = (d + inc) % 10
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)

# Load the MNIST dataset
num_classes = 10
(tr_pairs, tr_y), (te_pairs, te_y) = load_pairs(mnist, num_classes)

In [6]:
def initialize_base_network(input_shape):
    input = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu')(input)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    return Model(input, x)

def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

# Network definition
input_shape = tr_pairs[:, 0].shape[1:]
base_network = initialize_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

# Because we re-use the same instance `base_network`,
# the weights of the network will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model([input_a, input_b], distance)

In [7]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

history = model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y,
          batch_size=128,
          epochs=10,
          validation_data=([te_pairs[:, 0], te_pairs[:, 1]], te_y))

Epoch 1/10
847/847 [==============================] - 162s 190ms/step - loss: 0.7308 - accuracy: 0.4703 - val_loss: 0.7339 - val_accuracy: 0.4642
Epoch 2/10
847/847 [==============================] - 152s 179ms/step - loss: 0.7146 - accuracy: 0.4899 - val_loss: 0.7278 - val_accuracy: 0.4681
Epoch 3/10
847/847 [==============================] - 156s 184ms/step - loss: 0.7056 - accuracy: 0.5084 - val_loss: 0.7246 - val_accuracy: 0.4668
Epoch 4/10
847/847 [==============================] - 150s 177ms/step - loss: 0.6989 - accuracy: 0.5200 - val_loss: 0.7239 - val_accuracy: 0.4612
Epoch 5/10
847/847 [==============================] - 150s 177ms/step - loss: 0.6936 - accuracy: 0.5344 - val_loss: 0.7228 - val_accuracy: 0.4643
Epoch 6/10
847/847 [==============================] - 151s 178ms/step - loss: 0.6896 - accuracy: 0.5436 - val_loss: 0.7234 - val_accuracy: 0.4614
Epoch 7/10
847/847 [==============================] - 151s 178ms/step - loss: 0.6872 - accuracy: 0.5485 - val_loss: 0.7248 -

In [23]:
# Just to get something to test the newly trained network on quickly we reload data from MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Select an image for testing
# For simplicity, we're taking the first image from the test set
new_image_array = test_images[0]

# Siamese networks expect input data to have a specific shape, often including the channel dimension.
# MNIST images are 28x28 and grayscale, so we need to reshape them to 28x28x1.
new_image_array = np.expand_dims(new_image_array, axis=-1)

# Normalize the image
new_image_array = new_image_array.astype('float32') / 255.0

new_image_embedding = base_network.predict(np.array([new_image_array]))

print("Generated Embedding:", new_image_embedding)
new_image_embedding.shape # A vector of length 128

ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 219, in assert_input_compatibility
        raise ValueError(

    ValueErro

In [24]:
model.save('siamese_network.h5')  # saves the entire model to a HDF5 file
model.save_weights('siamese_network_weights.h5')  # saves just the model weights
base_network.save('base_network.h5') # saves just base network, for actually generating embedding rather than the full siamese architecture

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [25]:
from tensorflow.keras.models import load_model

loaded_model = load_model('siamese_network.h5') # load the entire model
model.load_weights('siamese_network_weights.h5') # If you've only saved the weights and want to load them, you'll need to define the model architecture first and then load the weights
loaded_base_network = load_model('base_network.h5') # loading the base network